<a href="https://colab.research.google.com/github/oliverguhr/deep-nlp-workshop/blob/main/Workshop_Sprache_ist_komplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gut oder schlecht? Ein simples Machine-Learning-Szenario.

Jeden Tag schrieben hunderte Kunden ihre Meinung zu unseren Produkten. Dabei den Überblick 
zu behalten, ist nicht einfach. Wir möchten daher ein kleines Programm schreiben, welches feststellt ob die 
Kunden positiv oder eher negativ über unsere Produkte sprechen.

Dazu bekommen wir eine Liste der Kundenmeinungen. Wir sollen ein kleines Prgramm schreiben, welches eine 
Eins zurückgibt wenn eine Kundenaussage positiv ist und eine Null zurück gibt, wenn eine Kundenaussage 
negativ ist. Auf diese Weise können wir am Schluss einfach auszählen wieviel Prozent der Kunden postiv über unsere Produkte sprechen.

In [1]:
#Empirische Daten sind "messy".

feedback_texte = ["Das ist ein tolles Produkt.",
                  "Ich bin super zufrieden!",
                  "Funktioniert nur sehr schlecht.",
                  "Ist mir viel zu teuer.",                                    
                  "Super Produkt, toller service",
                  "Für die Leistung zu teuer.",
                  "Ein ein super Produkt." #Tippfehler
                  "Super Preis, für die Leistung", #uneindeutige Aussage
                  "Hab eins für meine Oma gekauft und die ist sehr zufrieden",
                  "Kunderservice ist ein nur toll."] #auch hier eine uneindeutige Aussage

In [2]:
#Overfitting: Wenn wir unser Modell nur auf Grundlage obiger Daten bauen, könnten wir auf die Idee kommen, dass eine
#triviale Lösung gut genug ist. Für zuverlässig funktionierende Modelle brauchen wir ausreichende - und ausreichend
#diverse - Trainingsdaten. Andernfalls "overfitten" wir unser Modell auf eine nicht-repräsentative Teilmenge der Daten.

#Triviale Lösung: Unser Kunden-Feedback ist immer super.
def sentiment(text):
    return 1

for text in feedback_texte:
    print(f"{sentiment(text)} = {text}")

1 = Das ist ein tolles Produkt.
1 = Ich bin super zufrieden!
1 = Funktioniert nur sehr schlecht.
1 = Ist mir viel zu teuer.
1 = Super Produkt, toller service
1 = Für die Leistung zu teuer.
1 = Ein ein super Produkt.Super Preis, für die Leistung
1 = Hab eins für meine Oma gekauft und die ist sehr zufrieden
1 = Kunderservice ist ein nur toll.


## Mehr Texte

Da unser System so gut funktioniert, bitten uns die Kollegen weitere Texte zu analysieren:

In [3]:
#Vielleicht ist die triviale Lösung doch nicht so gut.
feedback_texte_advanced = ["Das ist ein dolles Produkt.", #Umgangssprache
                  "Macht meistens was es soll.", #uneindeutig
                  "Das Produkt ist gar nicht so toll wie ich gedacht hatte.", #uneindeutig
                  "super, schon nach ZWEI tagen kaputt!!!", #aussagekräftige Typographie                                    
                  "Bester Service der Welt. Nicht.", #Satzübergreifende Semantik
                  "Gute Leistung, der Preis ist aber nicht angemessen.",
                  "Gar nicht mal so gut.", #uneindeutig
                  "Super Preis, mieser Service.",
                  ":(", #Sonderzeichen
                  "1 nices teil"] #Umgangssprache

for text in feedback_texte_advanced:
    print(f"{sentiment(text)} = {text}")

1 = Das ist ein dolles Produkt.
1 = Macht meistens was es soll.
1 = Das Produkt ist gar nicht so toll wie ich gedacht hatte.
1 = super, schon nach ZWEI tagen kaputt!!!
1 = Bester Service der Welt. Nicht.
1 = Gute Leistung, der Preis ist aber nicht angemessen.
1 = Gar nicht mal so gut.
1 = Super Preis, mieser Service.
1 = :(
1 = 1 nices teil


## Machine Learning

Wir machen eine ganz einfache BOW-NB-Klassifikation.
(BOW - bag of words: wir klassifizieren die Texte anhand ihres Vokabulars, ignorieren aber die Reihenfolge der Wörter;
NB - Naive Bayes: eine simple, probabilistische Klassifikation)

In [4]:
#scikit-learn ist eine gut entwickelte Statistik- und ML-Bibliothek für Python
#Wir nutzen den TfidfVectorizer für die "feature extraction", also um den Text in eine ML-fähige Repräsentation zu bringen.
from sklearn.feature_extraction.text import TfidfVectorizer 
#Das ist unser Klassifikationsalgorithmus
from sklearn.naive_bayes import MultinomialNB

In [5]:
#Wir bereiten unsere Testdaten gleich mit vor. Die brauchen wir, um unser Modell zu validieren.
test_texte = ["Ich bin total zufrieden.",
       "Mann, ich bin SOWAS VON zufrieden!!!",
       "Würde ich wieder kaufen.",
       "Wirklich großer Mist.",
       "Ist schon kaputt",
       "Voll gut!"]

test_labels = [1,1,1,0,0,1]

vectorizer = TfidfVectorizer()
data = list()
data.extend(feedback_texte)
data.extend(feedback_texte_advanced)
data.extend(test_texte)

In [6]:
#Für ML-Experimente müssen wir Text i. d. R. in eine geeignete mathematische Repräsentation bringen.
#Hier repräsentieren wir den Datensatz als Matrix von Tfidf-Werten.

all_data = vectorizer.fit_transform(data)
#Wir wollen lernen, sentiment labels (1=positiv, 0=negativ) vorherzusagen. Dafür trainieren wir mit bereits gelabelten Daten.
#Sind die train_labels wirklich korrekt? Wie könnte man die Kundenmeinungen noch einordnen?
train_labels = [1,1,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,1]

#Wir splitten Trainings- und Testdaten.
train_data = all_data[:len(feedback_texte)+len(feedback_texte_advanced)]
test_data = all_data[len(feedback_texte)+len(feedback_texte_advanced):]

In [7]:
#Wir trainieren ein Modell.
#https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

nb = MultinomialNB()
model = nb.fit(train_data, train_labels)

In [8]:
#Wir testen das Modell, indem wir die Vorhersagen auf den Testdaten auswerten.

predictions = model.predict(test_data)

In [9]:
for text, prediction in zip(test_texte, predictions):
    print(text, prediction)

Ich bin total zufrieden. 1
Mann, ich bin SOWAS VON zufrieden!!! 1
Würde ich wieder kaufen. 0
Wirklich großer Mist. 0
Ist schon kaputt 0
Voll gut! 0


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(predictions, test_labels)

print(f'Unser Modell klassifiziert {accuracy*100} % der Sätze richtig. ')

Unser Modell klassifiziert 66.66666666666666 % der Sätze richtig. 
